In [1]:
import sys
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import math


# Homework 3 from Martin Gräf, Richard Baumann and Thomas Block

## Exercise 1
Derive corresponding expressions for O[φ] for  magnetization (per site) and energy (per site). 





## Exercise 2

### Determine the equations of motion (EoMs) for this Hamiltonian by applying the Hamilton equations.

The Hamilton operator of the problem is given as:

\begin{equation}
    H=+\dfrac{p^2}{2}+\dfrac{{\varphi}^2}{2{\beta}\hat{J}}-N\ln\left(2\cosh\left({\varphi}+{\beta}h\right)\right).
\end{equation}

The equations of motion are given as:

\begin{equation}
    \dot{\varphi} = \dfrac{\partial}{\partial p} H = p
\end{equation}
\begin{equation}
    \dot{p} = \dfrac{\partial}{\partial \varphi} H = 
    \dfrac{{\varphi}}{{\beta}\hat{J}}-\dfrac{N\sinh\left({\varphi}+{\beta}h\right)}{\cosh\left({\varphi}+{\beta}h\right)}
\end{equation}

These can be implemented as:

In [5]:
def phi_dot(p , phi, beta, J, h):
    # both p and phi are two-element arrays.  phi doesn't do anything
    return p

def p_dot(p, phi, beta, J, h):
    # both p and phi are two-element arrays.  p doesn't do anything
    first_term=pgi/(beta*J)
    second_term=N*math.sinh(phi+beta*h)/(math.cosh(phi+beta*h))
    return (first_term-second_term)

## Exercise 3

Code up the leapfrog algorithm to evolve (p0, φ0) → (pf , φf ). Verify the convergence claim H(pf , φf ) = H(p0, φ0) + O(2). You should find something similar to Figure 1
(especially for large Nmd)

In [3]:
def leapfrog(p_in, phi_in, Nmd, trajLength = 1):
    p_out = np.array(p_in)
    phi_out = np.array(phi_in)
    
    epsilon = trajLength/Nmd
    
    # first half step
    p_out = p_out + .5*epsilon*p_dot(p_out,phi_out)
    
    # now let the frogs jump. . .
    for i in range(Nmd):
        phi_out = phi_out + epsilon*phi_dot(p_out,phi_out)
        p_out = p_out + epsilon*p_dot(p_out,phi_out)
        
    # now we've gone one step too far
    p_out = p_out -  .5*epsilon*p_dot(p_out,phi_out)
    
    return p_out,
    
    

Testing the leapfrog

## Exercise 4
With your working leapfrog integrator, code up the HMC algorithm for the long-range Ising model.

## Exercise 5
Setting h = (βh) = .5 and using some values of N ranging from 5 to 20, calculate the average energy per site and mean magnetization per site as a function of J = (βJ) ∈ [.2, 2]. Tune Nmd such that the acceptance rate is above 50%